In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def bias_add(V):
    bias = -np.ones( (len(V),1) )
    return np.concatenate( (V,bias), axis=1)

def bias_sub(V):
    return V[:,:-1] # Saco la ultima columna

In [ ]:
def activation(X): 
    
    Y = [] # L capas Y_o, Y_1, Y_2, .... Y_l-1
    i = 0
    P = len(X)
    for s in S: # S variable global con los sizes de las capas 
        if i == len(S)-1:
            Y.append( np.zeros((P,s)) )
            i += 1
        else:
            Y.append( np.zeros((P,s+1)) )
            i += 1
        
    Y[0][:] = bias_add(X)
    
    for k in range(1,len(S)-1):
        Y[k][:] = bias_add( np.tanh( np.dot(Y[k-1],W[k]) ) ) # W global (se arman a parte)
        
    Y[-1][:] = np.tanh( np.dot(Y[-2],W[-1]) )
    return Y  

In [ ]:
def correction(Y,Z):
    delta_W = [0] # primer elemento vacio
    
    for i in range(1,len(S)): # S variable global con los sizes de las capas 
        delta_W.append( np.zeros( (S[i-1]+1,S[i]) ) )
        
    L = len(S) 
    E = [0]*L # size L 
    D = [0]*L # size L
    d = [0]*L # size L
    
    E[L-1] =  Z - Y[-1] 
    d[L-1] = 1 - Y[-1]**2
    D[L-1] = E[-1]*d[-1] 
        
    for k in reversed(np.arange(1,L)):
        delta_W[k] = lr * np.dot( Y[k-1].T, D[k]) # lr = learning rate (variable global)
        E[k] = np.dot( D[k], W[k].T )
        d[k] = ( 1 - Y[k-1]**2 )
        D[k-1] = ( bias_sub( E[k]*d[k] ) )
    return delta_W

In [ ]:
def adaptation(W,delta_W):
  L = len(S) # S Variable global
  for k in range(1,L-1):
    W[k] += delta_W[k]
  return W

In [ ]:
def estimation(Z,Y):
  est = np.mean( np.square(Z - Y) )
  return est

In [ ]:
def train(X, Z, S, W, lr, eps, epoch):

  error = 1
  iter = 0
  errores = []
  while error > eps and iter < epoch:

    Y = activation(X)
    delta_w = correction(Y,Z)
    W = adaptation(W,delta_w)
    error = estimation(Z,Y[-1])

    errores.append(error)
    iter += 1

  return errores

In [ ]:
def train_stochastic(X, Z, S, W, lr, eps, epoch):
    
  error = 1
  iter = 0
  errores = []
  P = len(X)
  while error > eps and iter < epoch:
    e = 0
    stochastic = np.random.permutation(P)
    for h in stochastic:
        Xh = X[h:h+1]
        Zh = Z[h:h+1]

        Y = activation(Xh)
        delta_w = correction(Y,Zh)
        W = adaptation(W,delta_w)
        e += np.sum( np.square( Z - Y[-1] ))
        
    error = e/P
    errores.append(error)
    iter += 1

  return errores